In [1]:
#Import Dependencies
from pickle import load
import pandas as pd
import numpy as np
from api_keys import g_key
import gmaps
import requests
import warnings
warnings.filterwarnings('ignore')

# Loading Models and Scalars

In [2]:
model = load(open('model_artifacts/model.pkl', 'rb'))
scalar = load(open('model_artifacts/scalar.pkl', 'rb'))

bonus_model = load(open('model_artifacts/bonus_model.pkl', 'rb'))
bonus_scalar = load(open('model_artifacts/bonus_scalar.pkl', 'rb'))

equity_model = load(open('model_artifacts/equity_model.pkl', 'rb'))
equity_scalar = load(open('model_artifacts/equity_scalar.pkl', 'rb'))


# Distance function calculates the distance of the job location from NY, SF and Seattle using Google APIs

In [3]:
def distances(origin_city, destination_cities, g_key):
    base_url = "https://maps.googleapis.com/maps/api/distancematrix/json?"
    distances=[]

    
    for i in range(len(destination_cities)):
        
        url=base_url+"origins="+origin_city+"&destinations="+destination_cities[i]+"%20C"+"&key="+g_key

        payload={}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)

        x=response.json()

        y=x["rows"][0]["elements"][0]["distance"]["value"]

        distances.append(y)
        
    return(distances)


# Prediction Function that calculates  the yearly compensation, bonus and equity using the scalars and models

In [4]:
def predict(yac, exp, level, d1, d2, d3):
    X_be=np.array([[level, exp, yac, d1, d2, d3]])
    X_be=X_be.reshape(1, -1)
    bonus_scaled=bonus_scalar.transform(X_be)
    bonus=bonus_model.predict(bonus_scaled)
    
    equity_scaled=equity_scalar.transform(X_be)
    equity=equity_model.predict(equity_scaled)
    
    X=np.array([[level, exp, yac, d1, bonus, equity, d2, d3]])
    X=X.reshape(1, -1)
    scaled=scalar.transform(X)
    salary=model.predict(scaled)
    
    Y=[bonus, equity, salary]
    return Y
 

# Takes in User Inputs and calls the prediction function

In [9]:
yac=input("Enter years at company(in years): ")
exp=input("Enter years of experience(in years): ")
city=input("Enter company location city: ")
state=input("Enter company location state abbreviation: ")
print("Enter your level. Eg. 10 for VP, 1 for Junior for Associate Data Scientist")
level=input("Enter level: ")

origin_city=city+"%20C"+state
destination_cities=["New York%20CNY","San Francisco%20CCA", "Seattle%20CWA"]

D_y=distances(origin_city, destination_cities, g_key)

result=predict(yac, exp, level, D_y[0], D_y[1], D_y[2])
bonus=result[0]
equity=result[1]
salary=result[2]


Enter years at company(in years): 0
Enter years of experience(in years): 10
Enter company location city: San Francisco
Enter company location state abbreviation: CA
Enter your level. Eg. 10 for VP, 1 for Junior for Associate Data Scientist
Enter level: 8


# Prints the total compensation, bonus and equity

In [10]:
print(f"Your expected total yearly compensation is: ${salary}")
print(f"Your expected bonus is: ${bonus}")
print(f"Your expected equity is: ${equity}")

Your expected total yearly compensation is: $[315.05406749]
Your expected bonus is: $[26.66141486]
Your expected equity is: $[100.17848505]


# Previously run results

0, 10, Lincoln, NE, 8            194, 26, 43
2, 10, Lincoln, NE, 8            184, 26, 34
5, 10, Lincoln, NE, 8            149, 26, 17
2, 5, Lincoln, NE, 5             112, 14, 14

0, 10, Chicago, IL, 8            200, 26, 38 
2, 10, Chicago, IL, 8            184, 26, 27
5, 10, Chicago, IL, 8            157, 26, 10
2, 5, Chicago, IL, 5             134, 14, 15

0, 10, San Francisco, CA, 8      315, 26, 100
2, 10, San Francisco, CA, 8      313, 26, 98
5, 10, San Francisco, CA, 8      304, 26, 92
2, 5, San Francisco, CA, 5       253, 17, 68

# Observed interesting trends

At any company you are more likely to get a larger salary if you are just starting

Cities, with greater opportunities for data science jobs  and/or with higher living expense are expected to have larger compensations.

For same number of years at a company, lower level and experience means lesser salary, whuch is intuitive.